In [1]:
import sys
print(sys.executable)


c:\Users\smanasi\AppData\Local\miniconda3\envs\mammal\envs\mammal310\python.exe


In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("../data/sim_dataTL/sim_magnav_denoised.csv")
print(df.columns)
print("rows:", len(df))
print("num lines:", df["line"].nunique() if "line" in df else df["line_id"].nunique())
df.head()


Index(['time_s', 'lat', 'lon', 'alt_m', 'mag_scalar_nT',
       'mag_anomaly_injected_nT', 'mag_residual_nT', 'heading_deg',
       'turn_score_deg', 'is_turn', 'LINE', 'east_m', 'north_m', 'line',
       'LINE_TYPE', 'yaw_rad', 'pitch_rad', 'roll_rad', 'yaw_deg', 'pitch_deg',
       'roll_deg', 'psi_deg', 'psi_rad', 'F_raw', 'F_headcorr', 'delta_F',
       'F_smooth_sg', 'F_smooth_med', 'F_smooth_lp'],
      dtype='object')
rows: 148016
num lines: 18


,time_s,lat,lon,alt_m,mag_scalar_nT,mag_anomaly_injected_nT,mag_residual_nT,heading_deg,turn_score_deg,is_turn,...,pitch_deg,roll_deg,psi_deg,psi_rad,F_raw,F_headcorr,delta_F,F_smooth_sg,F_smooth_med,F_smooth_lp
0,0.00,-27.116237,131.084472,100.0,55445.483869,-6.953404,35.695884,0.0,0.0,False,...,0.0,0.0,0.0,0.0,35.695884,27.892867,7.803018,1.004351,0.0,36.926694
1,0.01,-27.116235,131.084472,100.0,55382.321804,-6.683435,-27.465100,0.0,0.0,False,...,0.0,0.0,0.0,0.0,-27.465100,-35.268117,7.803018,1.311708,0.0,34.160102
2,0.02,-27.116233,131.084472,100.0,55415.597191,-6.412450,5.811368,0.0,0.0,False,...,0.0,0.0,0.0,0.0,5.811368,-1.991650,7.803018,1.618821,0.0,31.474021
3,0.03,-27.116232,131.084472,100.0,55378.849986,-6.140469,-30.934757,0.0,0.0,False,...,0.0,0.0,0.0,0.0,-30.934757,-38.737774,7.803018,1.925684,0.0,28.886707
4,0.04,-27.116230,131.084472,100.0,55332.821138,-5.867511,-76.962524,0.0,0.0,False,...,0.0,0.0,0.0,0.0,-76.962524,-84.765542,7.803018,2.232294,0.0,26.415056


In [3]:
F_col = "F_smooth_sg"   # change to your actual column name
df["F"] = df[F_col].astype(float)


In [4]:
survey_df = pd.DataFrame({
    "LAT": df["lat"].astype(float),
    "LONG": df["lon"].astype(float),
    "F": df["F"].astype(float),
    "LINE": df["LINE"].astype(int),
    "LINE_TYPE": df["LINE_TYPE"].astype(int),
})
